In [2]:
import tensorflow as tf
print(tf.__version__)


2.11.0


In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import clear_output
import six.moves.urllib as urllib

import tensorflow.feature_column as fc
import tensorflow as tf

# Note: Linear Regression is an example of supervised Learning 

# Inputting the Dataset
dftrain = pd.read_csv('./data/train.csv') 
dfeval = pd.read_csv('./data/eval.csv')

y_train =dftrain.pop('survived')
y_eval = dfeval.pop('survived')

dftrain.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [7]:
# Separate the Categorical columns from the Numerical Columns
CATEGORICAL_COLS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
NUMERIC_COLS = ['age', 'fare']

# blank list to store feature columns
feature_columns = []
for feature_name in CATEGORICAL_COLS:
    # gets the list of all unique values from a given feature column
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
    
for feature_name in NUMERIC_COLS:
    # same thing above happens here too
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
    
print(feature_columns)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [8]:
#  training the Model
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True , batch_size=32):
    def input_f(): # inner function to input the parameters
        # This create an object in the form of a dictionary with parameters inputted
        ds =tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            # randomize the dataset
            ds = ds.shuffle(1000)
            # Split the datasets into batchsizes and number of epochs to feed into the model
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds # this returns a batch of datasets
    return input_f # returns a function object for use

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

# Linear regression comes into play here:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn) # train
result = linear_est.evaluate(eval_input_fn)

clear_output()
# to understand the accuracy of the algorithm involved in the prediction of the model developed
print(result['accuracy'])
print(result)

0.7386364
{'accuracy': 0.7386364, 'accuracy_baseline': 0.625, 'auc': 0.8325374, 'auc_precision_recall': 0.79915595, 'average_loss': 0.47713423, 'label/mean': 0.375, 'loss': 0.47045627, 'precision': 0.6530612, 'prediction/mean': 0.40466475, 'recall': 0.64646465, 'global_step': 200}


In [9]:
# using this alg. to make a prediction of who's gonna survive the titanic
result = list(linear_est.predict(eval_input_fn))

# Predicting The rate of survival for each individual
print(dfeval.loc[1])
print(result[1]['probabilities'][1])


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/yk/tx8wdy0n7vn_khcwz18g1d8w0000gn/T/tmpv4wxic1w/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                          male
age                          54.0
n_siblings_spouses              0
parch                           0
fare                      51.8625
class                       First
deck                            E
embark_town           Southampton
alone                           y
Name: 1, dtype: object
0.41506225
